#### ckip


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
pip install -U ckiptagger[tfgpu,gdown]

     |████████████████████████████████| 411.0MB 31kB/s 
     |████████████████████████████████| 51kB 4.8MB/s 
     |████████████████████████████████| 512kB 43.9MB/s 
     |████████████████████████████████| 3.8MB 44.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=e470e69c1314a0128332b17e4567a62ff541faf98ea0e6394f327b3a1140fd7a
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.3.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorboard<3,>=2.3.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorflow-estimator<2.4.0,>=2.3.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing i

In [ ]:
import pandas as pd
import jieba
import jieba.posseg as pseg
import jieba.analyse
import numpy as np 

datapath = '/content/gdrive/My Drive/ai_news/'
jieba.set_dictionary(datapath + 'dict.txt.big')   #下載繁體中文詞庫後，使用set_dictionary()方法匯入


#載入停用字
with open(datapath+'stopword.txt','r',encoding='utf-8-sig') as file:
  stops = file.read().split('\n')
#載入姓氏
fn = pd.read_csv(datapath+'CFN_TN.txt', sep=" ",header=None)   


In [ ]:
from tensorflow import keras
MAX_NUM_WORDS = 10000
VALIDATION_RATIO = 0.1 
RANDOM_STATE = 9527   # 小彩蛋
NUM_CLASSES = 2      # 基本參數設置，有幾個分類
MAX_NUM_WORDS = 10000   # 在語料庫裡有多少詞彙
MAX_SEQUENCE_LENGTH = 250 # 一個標題最長有幾個詞彙
NUM_EMBEDDING_DIM = 256  # 一個詞向量的維度
NUM_LSTM_UNITS = 128   # LSTM 輸出的向量維度
tokenizer = keras.preprocessing.text.Tokenizer(num_words=MAX_NUM_WORDS)


In [ ]:
# max_seq_len = numpy.mean([len(seq) for seq in x_train])
# max_seq_len
df = pd.read_excel(datapath+'tnse_data.xlsx', encoding='utf-8' )
corpus = df.content
corpus = pd.concat([corpus])
corpus.shape
pd.DataFrame(corpus.iloc[:5],
             columns=['title'])
tokenizer.fit_on_texts(corpus)


In [ ]:

x_train = tokenizer.texts_to_sequences(corpus)

x_train = keras.preprocessing.sequence.pad_sequences(x_train,maxlen=MAX_SEQUENCE_LENGTH)

# 將分類標籤對應到剛定義的數字
y_train = df.label
y_train = np.asarray(y_train).astype('float32')
y_train = keras.utils.to_categorical(y_train)


In [ ]:
from sklearn.model_selection import train_test_split

  

x_train, x_val, y_train, y_val = train_test_split(  x_train,
                             y_train, 
                            test_size=VALIDATION_RATIO, 
                           random_state=RANDOM_STATE)

In [ ]:
# 建立孿生 LSTM 架構（Siamese LSTM）
from keras import Input
from keras.layers import Embedding,LSTM, concatenate, Dense
from keras.models import Model

ImportError: ignored

In [ ]:

# 分別定義 2 個新聞標題 A & B 為模型輸入兩個標題都是一個長度為 200 的數字序列
input = Input(shape=(MAX_SEQUENCE_LENGTH, ),dtype='int32')

# 詞嵌入層經過詞嵌入層的轉換，兩個新聞標題都變成一個詞向量的序列，而每個詞向量的維度為 256
embedding_layer = Embedding(MAX_NUM_WORDS, NUM_EMBEDDING_DIM)
embedded = embedding_layer(input)

# LSTM 層兩個新聞標題經過此層後為一個 128 維度向量
shared_lstm = LSTM(NUM_LSTM_UNITS)
output = shared_lstm(embedded)

# 全連接層搭配 Softmax Activation可以回傳 2 個成對標題屬於各類別的可能機率
dense =  Dense(
    units=NUM_CLASSES, 
    activation='softmax')
predictions = dense(output)  # 模型就是將數字序列的輸入，轉換成 3 個分類的機率的所有步驟 / 層的總和 

model = Model(
    inputs=[input], 
    outputs=predictions)   

model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy']) 

In [ ]:
BATCH_SIZE = 512  # 決定一次要放多少成對標題給模型訓練
NUM_EPOCHS = 10  # 決定模型要看整個訓練資料集幾遍

# 實際訓練模型
history = model.fit(
    x=x_train, # 輸入是兩個長度為 250 的數字序列
    y=y_train,
    batch_size=BATCH_SIZE,
    epochs=NUM_EPOCHS,# 每個 epoch 完後計算驗證資料集上的 Loss 以及準確度
    validation_data=(# 每個 epoch 隨機調整訓練資料集裡頭的數據以讓訓練過程更穩定
        x_val, 
        y_val
    ),
    shuffle=True
)

Train on 4379 samples, validate on 487 samples
Epoch 1/10
4379/4379 [==============================] - 41s 9ms/step - loss: 0.3902 - accuracy: 0.8838 - val_loss: 0.1930 - val_accuracy: 0.9281
Epoch 2/10
4379/4379 [==============================] - 40s 9ms/step - loss: 0.1477 - accuracy: 0.9381 - val_loss: 0.1368 - val_accuracy: 0.9302
Epoch 3/10
4379/4379 [==============================] - 40s 9ms/step - loss: 0.0948 - accuracy: 0.9546 - val_loss: 0.1178 - val_accuracy: 0.9487
Epoch 4/10
4379/4379 [==============================] - 40s 9ms/step - loss: 0.0739 - accuracy: 0.9705 - val_loss: 0.1099 - val_accuracy: 0.9363
Epoch 5/10
4379/4379 [==============================] - 41s 9ms/step - loss: 0.0386 - accuracy: 0.9849 - val_loss: 0.0848 - val_accuracy: 0.9589
Epoch 6/10
4379/4379 [==============================] - 40s 9ms/step - loss: 0.0264 - accuracy: 0.9918 - val_loss: 0.1078 - val_accuracy: 0.9466
Epoch 7/10
4379/4379 [==============================] - 40s 9ms/step - loss: 0.0213

In [ ]:
# model.save(datapath + '/modellstm_10.h5')

In [ ]:
del model

In [ ]:
model = keras.models.load_model(datapath + '/modellstm_10.h5')


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
model.evaluate(x_train,y_train)

4379/4379 [==============================] - 32s 7ms/sample - loss: 0.0407 - acc: 0.9863


[0.04068112633054722, 0.98629826]

### 以下是匯入測試資料

In [ ]:
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
import re
ws = WS(datapath+"/data")    #斷詞
pos = POS(datapath+"/data")   #詞性標記
ner = NER(datapath+"/data")   #命名實體識別

In [ ]:
import pandas as pd
test = pd.read_excel(datapath+'test.xlsx', encoding='utf-8' )
test
# test.drop(test[test.index>=2].index, inplace= True)

,Unnamed: 0,CONTENT,LINK
0,0,前有洗錢防制、後有反送中，金融圈人士指出，香港分行的財管業務恐怕癱瘓，其中，公股行庫的財管業...,https://www.chinatimes.com/newspapers/20200528...
1,1,金管會為使防制洗錢措施順利執行，今年請銀行公會及銀行研訓院合計舉辦5場「洗防RBA，好評又便...,https://www.chinatimes.com/realtimenews/201907...
2,2,內政部資訊中心主任施明德涉嫌利用任職移民署移民資訊組組長期間，承辦7項採購案收受回扣，檢廉2...,https://www.chinatimes.com/realtimenews/201904...
3,3,28歲男子高豐霖，在北市內湖科技園區以經營貿易公司為幌子，實際上從事地下匯兌洗錢，成員外出收...,https://www.chinatimes.com/newspapers/20190316...
4,4,國際洗錢防制機構針對數位通貨制定管理守則，要求加密通貨業者或交易所須分享客戶資料，這促使業者...,https://www.chinatimes.com/newspapers/20190924...
...,...,...,...
195,195,加拿大西部英屬哥倫比亞省政府資料指出，2018年有逾70億加元透過當地的經濟體系洗錢，這些被...,https://www.chinatimes.com/realtimenews/201905...
196,196,香港警方昨日聲稱查獲一宗洗錢案，案情涉及與反送中活動相關的網上籌款平台「星火同盟」，引起關切...,https://www.chinatimes.com/realtimenews/201912...
197,197,近年歐洲地區洗錢案件頻傳，令歐洲金融管理局（EBA）飽受批評。在法國及荷蘭大力鼓吹之下，歐盟...,https://www.chinatimes.com/newspapers/20191101...
198,198,\n\n立委陳明文在高鐵上不慎遺失300萬元，法務部次長蔡碧仲25日在立法院財委會接受質詢時...,https://www.chinatimes.com/realtimenews/201909...


In [ ]:
sentence = test.CONTENT[0]

ws_results = ws([sentence])

for word in ws_results:
  if word in stops:
    sent2 = "".join('')
  else:
    sent2 = " ".join(word)
    # print(word)
        
regex = re.compile('([^\s\w]|_)+')
sent2 = regex.sub('', sent2)
sent2 = ' '.join(sent2.split())
[sent2]

['前 有 洗錢 防制 後 有 反送 中 金融圈 人士 指出 香港 分行 的 財管 業務 恐怕 癱瘓 其中 公股 行庫 的 財管 業務 在 國銀 中 可 說 最早 收攤 很多 行庫 去年 的 財管 業務 的 新作 案件 不 是 個位數 就是 掛零 現在 加上 存款 撤出潮 財管 業務 陷入 巧婦 難為 無 米炊 主掌 海外 事務 的 銀行 主管 分析 先前 在 香港 分行 的 營運 困境 主要 包括 了 洗錢 防制 及 反 資 恐 更為 嚴格 的 要求 CRS 及 中 美 兩 國 同步 追稅 以及 對 所謂 專業 投資人 增加 更多 門檻 等 三 大 面向 導致 國銀 的 香港 分行 財富 管理 業務 越來越 難 作 去年 下 半 年 又 碰到 長達 半 年 的 反送 中 更 是 雪上加霜 對於 來自 洗錢 防制 CRS 追稅 以及 對 專業 投資人 及 金融 商品 買賣 趨於 嚴格 限制 等 三 大 項 阻力 某 國銀 香港 分行 主管 內部 評估 前 幾 年 財管 業務 或許 在 花費 500萬 港幣 投入 電腦 資訊 設備 的 建置 之下 就 能 損益 兩 平 但 從 香港 洗錢 防制 規格 升高 之後 可能 要 2千萬 港幣 是 過去 的 四 倍 因為 得 投入 大量 的 資金 來 購置 電腦 資訊 系統 配備 以 達成 監理 機關 在 洗錢 防制 打擊 資恐 等 諸如 KYC EDD 客戶 交易 目的 及 後續 資金 流向 等 最 新 要求 這些 法規 比 台灣 還要 嚴格']

In [ ]:
# 將詞彙序列轉為索引數字的序列
x_test = tokenizer.texts_to_sequences([sent2])

# 為數字序列加入 zero padding
x_test = keras.preprocessing.sequence.pad_sequences(x_test,maxlen=MAX_SEQUENCE_LENGTH)
 


In [ ]:
# 利用已訓練的模型做預測
predictions = model.predict([x_test])
keynum = np.argmax(predictions[0])
keynum

0

In [ ]:
keyman = set({})
if keynum == 1:
  pos_results = pos(ws_results)
  ner_results = ner(ws_results, pos_results)
  for name in ner_results[0]:
    if name[2]=='PERSON':
      if bool(re.match(r'[\u4E00-\u9FFF]{2,4}', name[3])):
        if name[3][0] in fn[0].values:
          keyman.add(name[3])

print(list(keyman))

[]
